In [ ]:
%load_ext autoreload
%autoreload 2

# A pronounced loss of tree crown foliation results in multi-year growth reduction

We train model to predict the growth of trees during a given period of 4-6 years based on the following features:

| feature            | description                                  | level |
|--------------------|----------------------------------------------|-------|
| diameter_end       | Diameter at the end of the period            | tree  |
| defoliation_max    | Maximum defoliation of the growth period     | tree  |
| defoliation_min    | Minimum defoliation of the growth period     | tree  |
| defoliation_mean   | Mean defoliation of the growth period        | tree  |
| defoliation_median | Median defoliation of the growth period      | tree  |
| social_class_min   | Minimum social class of the growth period    | tree  |
| plot_latitude      | Latitude of the plot                         | plot  |
| plot_longitude     | Longitude of the plot                        | plot  |
| plot_slope         | Slope of the plot                            | plot  |
| plot_orientation   | Orientation of the plot                      | plot  |
| plot_altitude      | Altitude of the plot                         | plot  |
| dep_ph             | Deposition pH                                | plot  |
| dep_cond           | Deposition conductivity                      | plot  |
| dep_k              | Deposition potassium (K)                     | plot  |
| dep_ca             | Deposition calcium (Ca)                      | plot  |
| dep_mg             | Deposition magnesium (Mg)                    | plot  |
| dep_na             | Deposition sodium (Na)                       | plot  |
| dep_n_nh4          | Deposition ammonium (NH4)                    | plot  |
| dep_cl             | Deposition chloride (Cl)                     | plot  |
| dep_n_no3          | Deposition nitrate (NO3)                     | plot  |
| dep_s_so4          | Deposition sulfate (SO4)                     | plot  |
| dep_alk            | Deposition alkalinity                        | plot  |
| dep_n_tot          | Deposition total nitrogen (N)                | plot  |
| dep_doc            | Deposition dissolved organic carbon (DOC)    | plot  |
| dep_al             | Deposition aluminium (Al)                    | plot  |
| dep_mn             | Deposition manganese (Mn)                    | plot  |
| dep_fe             | Deposition iron (Fe)                         | plot  |
| dep_p_po4          | Deposition phosphate (PO4)                   | plot  |
| dep_cu             | Deposition copper (Cu)                       | plot  |
| dep_zn             | Deposition zinc (Zn)                         | plot  |
| dep_hg             | Deposition mercury (Hg)                      | plot  |
| dep_pb             | Deposition lead (Pb)                         | plot  |
| dep_co             | Deposition cobalt (Co)                       | plot  |
| dep_ni             | Deposition nickel (Ni)                       | plot  |
| dep_cd             | Deposition cadmium (Cd)                      | plot  |
| dep_s_tot          | Deposition total sulfur (S)                  | plot  |
| dep_c_tot          | Deposition total carbon (C)                  | plot  |
| dep_n_org          | Deposition organic nitrogen (N)              | plot  |
| dep_p_tot          | Deposition total phosphorus (P)              | plot  |
| dep_cr             | Deposition chromium (Cr)                     | plot  |
| dep_n_no2          | Deposition nitrite (NO2)                     | plot  |
| ss_ph              | Soil solution pH                             | plot  |
| ss_cond            | Soil solution conductivity                   | plot  |
| ss_k               | Soil solution potassium (K)                  | plot  |
| ss_ca              | Soil solution calcium (Ca)                   | plot  |
| ss_mg              | Soil solution magnesium (Mg)                 | plot  |
| ss_n_no3           | Soil solution nitrate (NO3)                  | plot  |
| ss_s_so4           | Soil solution sulfate (SO4)                  | plot  |
| ss_alk             | Soil solution alkalinity                     | plot  |
| ss_al              | Soil solution aluminium (Al)                 | plot  |
| ss_doc             | Soil solution dissolved organic carbon (DOC) | plot  |
| ss_na              | Soil solution sodium (Na)                    | plot  |
| ss_n_nh4           | Soil solution ammonium (NH4)                 | plot  |
| ss_cl              | Soil solution chloride (Cl)                  | plot  |
| ss_n_tot           | Soil solution total nitrogen (N)             | plot  |
| ss_fe              | Soil solution iron (Fe)                      | plot  |
| ss_mn              | Soil solution manganese (Mn)                 | plot  |
| ss_al_labile       | Soil solution labile aluminium (Al)          | plot  |
| ss_p               | Soil solution phosphorus (P)                 | plot  |
| ss_cr              | Soil solution chromium (Cr)                  | plot  |
| ss_ni              | Soil solution nickel (Ni)                    | plot  |
| ss_zn              | Soil solution zinc (Zn)                      | plot  |
| ss_cu              | Soil solution copper (Cu)                    | plot  |
| ss_pb              | Soil solution lead (Pb)                      | plot  |
| ss_cd              | Soil solution cadmium (Cd)                   | plot  |
| ss_si              | Soil solution silicon (Si)                   | plot  |
| soph_avg_sdi       | Average species diversity index              | plot  |
| soph_avg_age       | Average age of the trees                     | plot  |
| soph_avg_temp      | Average temperature                          | plot  |
| soph_avg_precip    | Average precipitation                        | plot  |

Here we import key libraries used in the notebook (run `uv sync` and select the corresponding kernel if some of them are missing).

In [ ]:
# Main libraries
import shap
import numpy as np

# Plotting
import matplotlib.pyplot as plt
import seaborn as sns

# We will use Polars for data manipulation
import polars as pl

# Casting types from time to time to have a better autocompletion
from typing import cast

from models import (
    train_and_explain,
    ExperimentResults,
    Species,
)

In [ ]:
from data import prepare_data, load_data

df = load_data("spruce")
X, y = prepare_data(df)

print("all variables:", list(df.columns))

# Compare absolute growth, growth rate, and relative growth rate (to diameter)
df.select("growth", "growth_rate", "growth_rate_rel").describe()

We then optimize hyperparameters and train models for every species. We may also select here the type of grouping used by the K-fold validation.

In [ ]:
# Predict the mean of the distribution
group_col = "plot_id"
all_species: list[Species] = ["spruce", "pine", "beech", "oak"]
all_results: dict[str, ExperimentResults] = {}

for species in all_species:
    all_results[species] = train_and_explain(
        species, model_type="lasso", group_by=group_col
    )

## Evaluate model performance for each species

Here we compute the mean and standard deviation of the $R^2$ (coefficient of determination) for all species and folds.

In [ ]:
perf = pl.concat(
    [
        pl.from_dicts(results.performances).select(
            pl.lit(species).alias("species"),
            pl.first().cum_count().alias("fold"),
            "test_r2",
            "train_r2",
        )
        for species, results in all_results.items()
    ]
)

perf = (
    perf.group_by("species")
    .agg(
        pl.mean("test_r2").alias("mean_test_r2"),
        pl.std("test_r2").alias("std_test_r2"),
        pl.mean("train_r2").alias("mean_train_r2"),
        pl.std("train_r2").alias("std_train_r2"),
    )
    .select(
        "species",
        test=pl.col("mean_test_r2").round(3).cast(pl.Utf8)
        + " +/- "
        + pl.col("std_test_r2").round(2).cast(pl.Utf8),
        train=pl.col("mean_train_r2").round(3).cast(pl.Utf8)
        + " +/- "
        + pl.col("std_train_r2").round(2).cast(pl.Utf8),
    )
    .unpivot(index="species")
    .pivot(index="variable", on="species", values="value")
    .rename({"variable": "split"})
)

with pl.Config() as cfg:
    cfg.set_tbl_formatting("ASCII_MARKDOWN", rounded_corners=True)
    cfg.set_tbl_hide_column_data_types(True)

    print(f"Performance summary for {len(all_species)} species:")
    print(perf)

## Feature importance by species

Here we show how feature importance varies by species.

In [ ]:
features = all_results["spruce"].features
top_n = 15

feature_importances = (
    pl.from_dicts(
        [
            {
                "species": species,
                "fold": fold,
                **dict(
                    zip(
                        features,
                        np.absolute(results.shap_values[fold].values).mean(axis=0),
                    )
                ),
            }
            for species, results in all_results.items()
            for fold in range(5)
        ]
    )
    .unpivot(
        on=pl.selectors.exclude("species", "fold"),  # type: ignore
        index=["species", "fold"],
        variable_name="feature",
        value_name="shap",
    )
    .with_columns(pl.col("shap").mean().over("feature").alias("importance"))
)

# Define order of features
feature_order = (
    feature_importances.group_by("feature")
    .agg(pl.col("importance").mean().alias("importance"))
    .sort("importance", descending=True)["feature"]
    .to_list()
)

sns.catplot(
    feature_importances,
    x="shap",
    y="feature",
    hue="species",
    kind="bar",
    order=feature_order[:top_n],
)
plt.xlabel("Mean SHAP absolute value")
plt.ylabel("Feature")

In [ ]:
# Plot the top N features by species for a given fold
fold = 0  # Change this to plot for a different fold
for species, results in all_results.items():
    plt.figure()
    ax = shap.plots.bar(
        results.shap_values[fold],
        max_display=10,
        ax=plt.gca(),
        show=False,
    )
    ax.set_title(f"Feature importance for {species.capitalize()} (fold {fold})")

## Feature dependence plots

These plots show how the Shapley value varies as a function of feature value.

In [ ]:
from scipy.optimize import curve_fit


def plot_dependence(
    results: ExperimentResults,
    feature: str,
    fold: int = 0,
    fit_curve: bool = False,
    xlim: tuple[float, float] | None = None,
    ylim: tuple[float, float] | None = None,
    ax: plt.Axes | None = None,
):
    if ax is None:
        _, ax = plt.subplots(figsize=(8, 6))

    if xlim is None:
        xlim = (
            np.nanmin(results.X[results.get_indices(fold, "all"), feature]),
            np.nanmax(results.X[results.get_indices(fold, "all"), feature]),
        )
    if ylim is None:
        ylim = (
            np.nanmin(results.shap_values[fold][:, feature].values),
            np.nanmax(results.shap_values[fold][:, feature].values),
        )

    # Enlarge slightly the limits for better visibility
    xlim = (xlim[0] - 0.05 * (xlim[1] - xlim[0]), xlim[1] + 0.05 * (xlim[1] - xlim[0]))
    ylim = (ylim[0] - 0.05 * (ylim[1] - ylim[0]), ylim[1] + 0.05 * (ylim[1] - ylim[0]))

    # Get the SHAP values for the selected feature
    shapley_values = cast(np.ndarray, results.shap_values[fold][:, feature].values)
    feature_values = results.X[results.get_indices(fold, "all"), feature].to_numpy()
    growth_values = results.y_true[results.get_indices(fold, "all")].to_numpy()

    # Order dataset by feature values
    order_idx = np.argsort(feature_values)
    feature_values = feature_values[order_idx]
    shapley_values = shapley_values[order_idx]
    growth_values = growth_values[order_idx]

    shap.plots.scatter(
        results.shap_values[fold][:, feature],
        # color=results.metadata[results.get_indices(fold, "all")]
        # .select(pl.col("plot_id").cast(pl.Categorical).to_physical())
        # .to_numpy().flatten(),
        alpha=0.4,
        ax=ax,
        show=False,
    )

    # Draw the line that indicates no effect
    ax.axhline(0, color="grey", linestyle="--")
    ax.text(xlim[1], ylim[1] / 20, "No effect", color="grey", ha="right")

    if fit_curve:
        # Fit a power law with vertical offset
        def func(x, a, b, c):
            return a * x**b + c

        # Get the SHAP values for the selected feature
        popt, _ = curve_fit(func, feature_values, shapley_values)

        # Plot the curve
        x = np.linspace(feature_values.min(), feature_values.max(), 100)
        ax.plot(
            x,
            func(x, *popt),
            color="red",
            label=f"y = {popt[0]:.2e} x^{popt[1]:.2f} + {popt[2]:.2f}",
        )

    ax.set_title(species.capitalize())
    ax.set_xlabel(feature)
    ax.set_ylabel("SHAP value")
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)

    return ax

In [ ]:
# Produce dependence plots for some key (species, feature) pairs
fold = 0  # Change this to plot for a different fold

# Beech: dependence on total nitrogen deposition
ax = plot_dependence(all_results["beech"], feature="dep_n_tot", fold=fold)
ax.set_title(f"Beech: Dependence of growth on total nitrogen deposition (fold {fold})")
ax.set_xlabel("Total nitrogen deposition [mg/l]")
ax.set_ylabel("SHAP value")

# Pine: dependence on deposition of phosphate
ax = plot_dependence(all_results["pine"], feature="dep_s_so4", fold=fold)
ax.set_title(
    f"Spruce: Dependence of growth on total phosphate deposition (fold {fold})"
)
ax.set_xlabel("Phosphate deposition [mg/l]")
ax.set_ylabel("SHAP value")

# Pine: dependence on iron deposition
ax = plot_dependence(all_results["pine"], feature="dep_fe", fold=fold)
ax.set_title(f"Spruce: Dependence of growth on iron deposition (fold {fold})")
ax.set_xlabel("Iron deposition [mg/l]")
ax.set_ylabel("SHAP value")

# Spruce: dependence on plot orientation
ax = plot_dependence(all_results["spruce"], feature="plot_orientation", fold=fold)
ax.set_title(f"Spruce: Dependence of growth on plot orientation (fold {fold})")
ax.set_xlabel("Plot orientation")
ax.set_ylabel("SHAP value")
ax.set_xticks(np.arange(9))  # 8 orientations + flat
ax.set_xticklabels(
    [
        "North",
        "North-east",
        "East",
        "South-east",
        "South",
        "South-west",
        "West",
        "North-west",
        "Flat",
    ],
    rotation=90,
)  # Adjust labels for better readability
ax.set_xlim(-0.5, 8.5)

# Oak: dependence on deposition pH
ax = plot_dependence(all_results["oak"], feature="dep_ph", fold=fold)
ax.set_title(f"Oak: Dependence of growth on deposition pH (fold {fold})")
ax.set_xlabel("Deposition pH")
ax.set_ylabel("SHAP value")

# Beech: dependence on plot slope
ax = plot_dependence(all_results["beech"], feature="plot_slope", fold=fold)
ax.set_title(f"Beech: Dependence of growth on plot slope (fold {fold})")
ax.set_xlabel("Plot slope")
ax.set_ylabel("SHAP value")

In [ ]:
feature = "defoliation_mean"  # Change this to the feature you want to plot
interacting = "dep_fe"
fit_curve = False
fold = 0

# One plot per species (4 in total)
fig, axes = plt.subplots(2, 2, figsize=(12, 8))

for species, ax in zip(all_species, axes.flatten()):
    plot_dependence(
        all_results[species],
        feature=feature,
        fold=fold,
        fit_curve=fit_curve,
        ax=ax,
    )
    ax.set_title(species.capitalize())
    ax.set_xlabel("Mean defoliation [%]")

if fit_curve:
    fig.suptitle(f"Dependence plots for {feature} with power law fit (fold {fold})")
else:
    fig.suptitle(f"Dependence plots for {feature} (fold {fold})")

plt.tight_layout()

# Species-specific plots

Hereafter we investigate a specific species and feature.

In [ ]:
species: Species = "spruce"
results = all_results[species]

## Feature interactions

We visualize the interaction between features by plotting two features against one each other.

In [ ]:
feature = "defoliation_mean"  # Feature to plot
# interacting = "defoliaton"

# Construct a DataFrame of the interaction values
feature_idx = results.X.columns.index(feature)

# Set axes
fig, ax = plt.figure(figsize=(6, 4)), plt.gca()

# Plot dependence plot with specific interaction
shap.plots.scatter(
    results.shap_values[fold][:, feature],
    # color=results.shap_values[fold][:, interacting].data,
    color=results.shap_values[fold],
    alpha=0.4,
    ax=ax,
    show=False,
)
plt.xlabel("Deposition of total nitrogen [mg / l]")
plt.ylabel("SHAP value")

# Set label of colorbar
ax.collections[0].colorbar.set_label("Tree Diameter [cm]")

In [ ]:
from explain import plot_ceteris_paribus_profile

fold = 0

X, _, y_pred = results.get_data(fold, "test")

# Plot 4 profiles for the selected feature
fig, axes = plt.subplots(2, 2, figsize=(10, 8))

y_vec = y_pred.to_numpy()

for i, ax in enumerate(axes.flat):
    if i == 0:
        # Take 5 growth rates in the 5th percentile
        low, high = np.min(y_vec), np.percentile(y_vec, 5)
    elif i == 1:
        # Take 5 growth rates in the [20, 30] percentile
        low, high = (
            np.percentile(y_vec, 20),
            np.percentile(y_vec, 35),
        )
    elif i == 2:
        # Take 5 growth rates in the [70, 80] percentile
        low, high = (
            np.percentile(y_vec, 70),
            np.percentile(y_vec, 80),
        )
    else:
        # Take 5 growth rates in the 95th percentile
        low, high = np.percentile(y_vec, 95), np.max(y_vec)

    plot_period_indices = np.random.choice(
        np.argwhere((y_vec >= low) & (y_vec < high)).flatten(),
        5,
    )
    title = f"Growth rates between [{low:.2f}, {high:.2f}]"

    for idx in plot_period_indices:
        feature_range, y_pred = plot_ceteris_paribus_profile(
            results.estimators[fold], X, idx, feature, ax=ax
        )
    ax.set_title(title)
    # ax.set_ylim([0.2, 0.8])

plt.tight_layout()

## Explain a subset of the data

We explain only data within:

- A given climatic area (`boreal`, `high_altitude`, `dry`).

In [ ]:
from data import load_data

split = "all"
min_samples = 250

# Load the data and the corresponding results
data = load_data(species)
results = all_results[species]

# Map each category to a Polar expression to retrieve the corresponding indices
category_idx_expr = {
    "boreal": pl.arg_where(pl.col("plot_latitude") >= 500000),
    "temperate": pl.arg_where(pl.col("plot_latitude") < 500000),
    "high_altitude": pl.arg_where(pl.col("plot_altitude") >= 1000),
    "low_altitude": pl.arg_where(pl.col("plot_altitude") <= 200),
    "hot": pl.arg_where(pl.col("soph_avg_temp") >= 13.5),
    "cold": pl.arg_where(pl.col("soph_avg_temp") <= 10),
    "flat": pl.arg_where(pl.col("plot_slope") <= 5),
    "hilly": pl.arg_where(pl.col("plot_slope") > 20),
    "dry": pl.arg_where(pl.col("soph_avg_precip") < 400),
    "humid": pl.arg_where(pl.col("soph_avg_precip") > 800),
}

shap_values = []
categories = []

for label, filter in category_idx_expr.items():
    indices = data.select(filter).to_series().to_numpy()

    if len(indices) < min_samples:
        print(f"Not enough samples for {label}")
        continue

    features = results.X[indices, :].to_pandas()
    shap_np = cast(np.ndarray, results.shap_values[fold][indices].values)

    for idx in range(shap_np.shape[0]):
        shap_values.append(shap_np[idx, :])

    categories.extend([label] * shap_np.shape[0])

# Plot the feature importances by category
df = (
    pl.from_numpy(np.absolute(np.stack(shap_values, axis=1).T), schema=results.features)
    .with_columns(pl.Series("condition", categories))
    .unpivot(
        on=pl.selectors.exclude("condition"),
        variable_name="feature",
        value_name="importance",
        index=["condition"],
    )
    .filter(pl.col("condition").is_in(["boreal", "high_altitude", "dry", "hot"]))
)

sns.catplot(
    df,
    x="importance",
    y="feature",
    hue="condition",
    kind="bar",
    order=feature_order[:10],
)
plt.xlabel("Feature importance (mean absolute SHAP value)")
plt.ylabel("Feature")
plt.title(f"Feature importances by condition (species = {species.capitalize()})")

In [ ]:
# Plot dependence plots for the selected feature by category
feature = "diameter_end"
interacting = ""

# 2x2 grid for the 4 categories
fig, axes = plt.subplots(2, 2, figsize=(12, 8))
axes = axes.flatten()

# Define x- and y-axis limits based on global min and max values
x_min = data[feature].min()
x_max = data[feature].max()
y_min = np.min(results.shap_values[fold][:, feature].values)
y_max = np.max(results.shap_values[fold][:, feature].values)

# Background points (temperate)
background_indices = data.select(category_idx_expr["temperate"]).to_series().to_numpy()

for idx, conditions in enumerate(["boreal", "high_altitude", "dry", "hot"]):
    indices = data.select(category_idx_expr[conditions]).to_series().to_numpy()

    if len(indices) < 25:
        print(f"Not enough samples for {conditions}")
        continue

    ax = axes[idx]

    shap.plots.scatter(
        results.shap_values[fold][background_indices, feature],
        ax=ax,
        alpha=0.05,
        show=False,
        hist=False,
        color="grey",
    )

    shap.plots.scatter(
        results.shap_values[fold][indices, feature],
        ax=ax,
        alpha=0.4,
        show=False,
        # Iterate on tab10 colors
        color=plt.cm.tab10(idx),
    )

    # Draw the line that indicates no effect with a label
    ax.axhline(0, color="grey", linestyle="--")
    ax.text(x_max, y_max / 20, "No effect", color="grey", ha="right")

    ax.set_title(conditions.capitalize().replace("_", " "))
    ax.set_xlabel(feature)
    ax.set_ylabel("SHAP values")
    ax.set_xlim(x_min, x_max)
    ax.set_ylim(y_min, y_max)

plt.tight_layout()

In [ ]:
# Visualize Shapley values using t-SNE
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE

# Build a dataframe of shapley values across all species
df_shap = []
species_col = []
plot_id_col = []
for species, results in all_results.items():
    for fold in range(5):
        df_shap.append(results.shap_values[fold].values)
        species_col.extend([species] * len(df_shap[-1]))
        plot_id_col.extend(results.metadata["plot_id"].to_numpy())

# Build a dataframe of SHAP values, downsampled to 20,000 samples
df_shap = (
    pl.from_numpy(np.concatenate(df_shap, axis=0), schema=results.features)
    .with_columns(pl.Series("species", species_col), pl.Series("plot_id", plot_id_col))
    .select("species", "plot_id", pl.exclude("species", "plot_id"))
).sample(n=20000)

# Standardize the data
scaler = StandardScaler()
X_shap = np.nan_to_num(
    scaler.fit_transform(df_shap.select(pl.exclude("species")).to_numpy())
)

# Train the t-SNE model
tsne = TSNE(n_components=2, perplexity=100, early_exaggeration=20)
X_tsne = tsne.fit_transform(X_shap)

In [ ]:
# Use seaborn to generate a scatter plot of the t-SNE results

ax = sns.scatterplot(
    data=df_shap.with_columns(
        pl.Series("tsne_x", X_tsne[:, 0]), pl.Series("tsne_y", X_tsne[:, 1])
    ).to_pandas(),
    x="tsne_x",
    y="tsne_y",
    hue="species",
    alpha=0.5,
    palette="muted",
    s=5,
    legend="brief",
)

ax.set_xlabel("Component 1")
ax.set_ylabel("Component 2")

plt.tight_layout()

In [ ]:
# Produce a zoom in a specific region
x_bounds = [0, 40]
y_bounds = [-40, 0]

# Create 2 subplots of the zoomed region: one for the species and one for the plot_id

fig, axes = plt.subplots(1, 2, figsize=(12, 6))

for ax, col in zip(axes, ["species", "plot_id"]):
    sns.scatterplot(
        data=df_shap.with_columns(
            pl.Series("tsne_x", X_tsne[:, 0]), pl.Series("tsne_y", X_tsne[:, 1])
        ).to_pandas(),
        x="tsne_x",
        y="tsne_y",
        hue=col,
        alpha=0.9,
        palette="muted" if col == "species" else "dark",
        legend=False,
        s=10,
        ax=ax,
    )

    ax.set_xlim(x_bounds)
    ax.set_ylim(y_bounds)

    ax.set_xlabel("Component 1")
    ax.set_ylabel("Component 2")

    plt.tight_layout()

## Feature interactions

We can compute the Shapley interactions using `get_shap_interactions`, which returns a tensor `(# samples, # features, # features)`, where each slice along the first axis is a symmetric matrix of interaction values, whose each row sums to the Shapley value for this feature. The diagonal entries represent the "main effect" attributed to that feature, whereas other entires represent the first-order interactions with every other feature.

In [ ]:
from explain import plot_interaction_matrix

top_n_features = (
    feature_importances.select("feature", "importance")
    .unique()
    .sort("importance", descending=True)
    .head(20)["feature"]
    .to_list()
)

interactions = {}
indices = {}

# Loop over the species
for species in all_species:
    results = all_results[species]

    # Plot the mean absolute interaction values for the selected feature as a heatmap
    plt.figure(figsize=(10, 8))
    ax = plt.gca()

    interactions[species], indices[species] = plot_interaction_matrix(
        results, top_n=top_n_features, ax=ax, vmax=0.005
    )

    plt.tight_layout()
    plt.title(f"Interactions for {species}")

    plt.savefig(f"figures/{species}-interactions-mean.png")

In [ ]:
interactions_matrix = interactions[species]
interactions_matrix.shape

np.vectorize(lambda m: m / np.sum(m, axis=1), signature="(m,m)->(m,m)")(
    interactions_matrix
)

In [ ]:
import networkx as nx

# Find all pair of features with a significant interaction
species = "spruce"
cutoff = 3e-3

results = all_results[species]
interactions_matrix = interactions[species]

interactions_matrix = np.absolute(interactions_matrix).mean(axis=0)
adjacency_matrix = np.triu(interactions_matrix, k=1)
adjacency_matrix[adjacency_matrix < cutoff] = 0.0

# Build graph from adjacency matrix
G = cast(
    nx.Graph,
    nx.from_numpy_array(
        adjacency_matrix,
        edge_attr="interaction",
        nodelist=results.features,
    ),
)

# Trim nodes without any connection
G.remove_nodes_from(list(nx.isolates(G)))

# Display graph with a circular layout
plt.figure(figsize=(12, 12))
pos = nx.circular_layout(G)

# Draw edges with a width proportional to the interaction strength
nx.draw_networkx_edges(
    G, pos, width=[G[u][v]["interaction"] * 1000 for u, v in G.edges()]
)

# Draw labels for the edges up in scientific notation with 2 decimal places
edge_labels = {
    k: f"{v:.2e}" for k, v in nx.get_edge_attributes(G, "interaction").items()
}
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels)

# Draw labels for the nodes while avoiding overlap
nx.draw_networkx_labels(
    G,
    pos,
    font_size=12,
    font_color="black",
    bbox=dict(facecolor="lightblue", boxstyle="round,pad=0.5,rounding_size=0.5"),
)
plt.title(f"Interaction graph for {species}")

In [ ]:
# Produce a plot to illustrate the interaction between two features
feature_idx = results.features.index("dep_fe")
interacting_idx = results.features.index("dep_p_po4")

plt.figure(figsize=(12, 8))

# Plot the interaction values for the selected feature
plt.scatter(
    results.X[indices[species], feature_idx].to_numpy(),
    results.X[indices[species], interacting_idx].to_numpy(),
    c=interactions[species][:, feature_idx, interacting_idx],
    cmap="coolwarm",
    alpha=0.5,
)

# Set labels for the x- and y-axis
plt.xlabel("Deposition of nitrogen")
plt.ylabel("Deposition of iron")

# Set log scale for the x- and y-axis
plt.xscale("log")
plt.yscale("log")


# plt.scatter()